Attribute Information:
> 1. age
> 2. sex
> 3. chest pain type (4 values)
> 4. resting blood pressure
> 5. serum cholestoral in mg/dl
> 6. fasting blood sugar > 120 mg/dl
> 7. resting electrocardiographic results (values 0,1,2)
> 8. maximum heart rate achieved
> 9. exercise induced angina
> 10. oldpeak = ST depression induced by exercise relative to rest
> 11. the slope of the peak exercise ST segment
> 12. number of major vessels (0-3) colored by flourosopy
> 13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
The names and social security numbers of the patients were recently removed from the database, replaced with dummy values. One file has been "processed", that one containing the Cleveland database. All four unprocessed files also exist in this directory.

Inspiration
Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0).

See if you can find any other trends in heart data to predict certain cardiovascular events or find any clear indications of heart health.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier


from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import OneHotEncoder

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_validate



In [ ]:
heart = pd.read_csv('../input/heart-disease-uci/heart.csv')

In [ ]:
heart.head(10)

In [ ]:
heart.info()

In [ ]:
heart.describe()

In [ ]:
heart.hist(figsize=(20,20))

In [ ]:
heart.skew()

In [ ]:
(np.log1p(heart)).skew()

In [ ]:
heart.columns

In [ ]:
sns.boxplot(x="cp", y ='target', data=heart )

In [ ]:
sns.countplot(x="cp", data=heart )

In [ ]:
sns.boxplot(data=heart, x='sex',y= 'target')

In [ ]:
sns.distplot(heart['sex'])

In [ ]:
sns.distplot(heart['cp'])

In [ ]:
heart.corr()['target']

In [ ]:
#Correlation Matrix
fig = plt.figure(figsize=[15,15])
plt.title('Pearson Correlation of Features', y=1.05, size=15)

sns.heatmap(heart.corr(), annot = True, square=True,linecolor='white',cmap='coolwarm' )

In [ ]:
heart.tail()

In [ ]:
y = heart['target']


In [ ]:
heart.sample(10)

In [ ]:
heart.nunique()

In [ ]:
hea = heart.copy()

In [ ]:
hea_dummy = pd.get_dummies(data=hea, columns= ['cp','fbs','restecg','exang','slope','ca','thal'], drop_first=True)

In [ ]:
hea_dummy.head()

In [ ]:
hea_dummy.columns

In [ ]:
scaler = StandardScaler()
X_new_dummy = scaler.fit_transform(hea_dummy.drop(columns='target'))

In [ ]:
selector = SelectFromModel(estimator=RandomForestRegressor(n_jobs=-1, n_estimators=100)).fit(X_new_dummy, y)
X_new_dummy = selector.transform(X_new_dummy)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new_dummy, y, test_size=0.2, random_state=42)

In [ ]:
model ={'clf':DecisionTreeClassifier(max_depth=3,min_samples_leaf =10, random_state=0),
       'Random Forest Classifier': RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0),
       'Logistic Regression': LogisticRegression(random_state=0),
      'Gaussian Naive': GaussianNB() }

In [ ]:
for keys, items in model.items():
   # print(f"key {keys}\n")
   # print( items)
    print(f"cross validation scores: {keys.upper()} ")
    print(cross_val_score(items, X_new_dummy, y, cv=5),"\n")
    

In [ ]:
model ={'clf':DecisionTreeClassifier(max_depth=3,min_samples_leaf =10, random_state=0),
       'Random Forest Classifier': RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0),
       'Logistic Regression': LogisticRegression(random_state=0),
      'Gaussian Naive': GaussianNB() }

for keys, items in model.items():
    cv_results = cross_validate(items, X_new_dummy, y, cv=5, scoring=('r2', 'f1','precision','recall','roc_auc'))
    print("keys" + "  " + str(keys),"\n")
    print("Recall:  ", cv_results['test_recall'])
    print("Precision:  ", cv_results['test_precision'],"\n")
    print("AUC:  ", cv_results['test_roc_auc'])
    print("max AUC:  ", max(cv_results['test_roc_auc']),"\n")
    